In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/1281081789.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92635,154673,https://www.slowboring.com/p/how-the-green-new...,How the Green New Deal became the Inflation Re...,\N,\N,2022-08-02 15:13:57.987661+00,2022-08-02 15:13:57.987661+00,NaN,\N,Yglesias,94.0,0.0,\N
92636,154674,https://www.askamanager.org/2022/08/my-younger...,my younger employee doesn’t know professional ...,\N,\N,2022-08-02 15:13:57.989217+00,2022-08-02 15:13:57.989217+00,NaN,\N,AskManager,9.0,0.0,\N
92637,154675,https://forum.effectivealtruism.org/posts/me9F...,Meditation course claims 65% enlightenment rat...,\N,\N,2022-08-02 15:13:57.990705+00,2022-08-02 15:13:57.990705+00,NaN,\N,EAForum,45.0,0.0,\N
92638,154676,https://split-ticket.org/2022/08/01/is-it-norm...,Is It Normal For Young Voters To Be This Democ...,\N,\N,2022-08-02 15:13:57.99232+00,2022-08-02 15:13:57.99232+00,NaN,\N,SplitTicket,30.0,0.0,\N


In [2]:
links['added'].max()

'2022-08-02 15:13:57.993841+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
92575    154611
92576    154612
92577    154613
92578    154614
92579    154615
Name: id, Length: 9688, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
92618,154656,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-08-02 15:13:57.961669+00,2022-08-02 15:13:57.961669+00,NaN,\N,CEA,90.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

92618    154656
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)


...1/83
...11/83


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)


...21/83
...31/83


...

...41/83
...51/83


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...61/83
...71/83
...81/83
...83/83


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_14926/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92718,154756,https://twitter.com/ohlennart/status/155374899...,"(1) Lennart Heim on Twitter: ""Also the video o...",\N,\N,2022-08-02 20:40:51.705239,2022-08-02 20:40:51.705252,\N,\N,Custom,2.0,0.0,NaN
92719,154757,https://twitter.com/JacobPeacock1/status/15533...,"(1) Jacob Peacock on Twitter: ""Protesting ""ven...",\N,\N,2022-08-02 20:40:51.715965,2022-08-02 20:40:51.715972,\N,\N,Custom,84.0,0.0,NaN
92720,154758,https://twitter.com/otis_reid/status/155370069...,"(1) Otis Reid on Twitter: ""It’s cool that we h...",\N,\N,2022-08-02 20:40:51.727562,2022-08-02 20:40:51.727573,\N,\N,Custom,46.0,0.0,NaN
92721,154759,https://twitter.com/MTabarrok/status/155373581...,"(1) Maxwell Tabarrok 🏗️🚀 on Twitter: ""If progr...",\N,\N,2022-08-02 20:40:51.739278,2022-08-02 20:40:51.739290,\N,\N,Custom,72.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92718,154756,https://twitter.com/ohlennart/status/155374899...,"(1) Lennart Heim on Twitter: ""Also the video o...",\N,\N,2022-08-02 20:40:51.705239,2022-08-02 20:40:51.705252,\N,\N,Custom,2.0,0.0,NaN
92719,154757,https://twitter.com/JacobPeacock1/status/15533...,"(1) Jacob Peacock on Twitter: ""Protesting ""ven...",\N,\N,2022-08-02 20:40:51.715965,2022-08-02 20:40:51.715972,\N,\N,Custom,84.0,0.0,NaN
92720,154758,https://twitter.com/otis_reid/status/155370069...,"(1) Otis Reid on Twitter: ""It’s cool that we h...",\N,\N,2022-08-02 20:40:51.727562,2022-08-02 20:40:51.727573,\N,\N,Custom,46.0,0.0,NaN
92721,154759,https://twitter.com/MTabarrok/status/155373581...,"(1) Maxwell Tabarrok 🏗️🚀 on Twitter: ""If progr...",\N,\N,2022-08-02 20:40:51.739278,2022-08-02 20:40:51.739290,\N,\N,Custom,72.0,0.0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92718,154756,https://twitter.com/ohlennart/status/155374899...,"(1) Lennart Heim on Twitter: ""Also the video o...",\N,\N,2022-08-02 20:40:51.705239,2022-08-02 20:40:51.705252,\N,\N,Custom,2,0,NaN
92719,154757,https://twitter.com/JacobPeacock1/status/15533...,"(1) Jacob Peacock on Twitter: ""Protesting ""ven...",\N,\N,2022-08-02 20:40:51.715965,2022-08-02 20:40:51.715972,\N,\N,Custom,84,0,NaN
92720,154758,https://twitter.com/otis_reid/status/155370069...,"(1) Otis Reid on Twitter: ""It’s cool that we h...",\N,\N,2022-08-02 20:40:51.727562,2022-08-02 20:40:51.727573,\N,\N,Custom,46,0,NaN
92721,154759,https://twitter.com/MTabarrok/status/155373581...,"(1) Maxwell Tabarrok 🏗️🚀 on Twitter: ""If progr...",\N,\N,2022-08-02 20:40:51.739278,2022-08-02 20:40:51.739290,\N,\N,Custom,72,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)